In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('vk-crawl.csv', sep=',')

df = df.dropna()

df = df[df['text'].str.contains('[a-zA-Zа-яА-Я]')]

df = df.sample(frac=1).reset_index(drop=True)
idxs = []
for a in range(16,41):
    idxs.extend(df.index[df['age'] == a].tolist()[:4000])
df = df.loc[idxs]

df = df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head()

(99971, 6)


,id,text,sex,city,age,source
0,55252596,https://vk.com/club142720150,2,1,26,1
1,428254633,Что нужно смертный?,2,1,27,1
2,300050466,ухожу в фейсбук,2,1,39,1
3,213834351,Хочешь зарабатывать на просмотрах рекламы? Спр...,1,1,22,1
4,139914748,"Приятно осознавать, что зависимость взаимна!!!...",1,2,33,1


In [3]:
corpus = df['text'].values
vect = TfidfVectorizer(ngram_range=(1,1), min_df=30)
X = vect.fit_transform(corpus)
X = X.toarray()
print(X.shape)

(99971, 2567)


In [15]:
y_sex = df['sex'].values - 1
print(y_sex.shape)
y_age = np.log(df['age'].values) - np.log(16) ## log
print(y_age.shape)

(99971,)
(99971,)


In [16]:
n_samples = X.shape[0]
frac = 0.8
X_train = X[:int(n_samples*frac)]
X_test = X[int(n_samples*frac):]
y_sex_train = y_sex[:int(n_samples*frac)]
y_sex_test = y_sex[int(n_samples*frac):]
y_age_train = y_age[:int(n_samples*frac)]
y_age_test = y_age[int(n_samples*frac):]

In [26]:
sex_predictor = LogisticRegression()

sex_predictor.fit(X_train, y_sex_train)
y_sex__ = sex_predictor.predict(X_test)

print('Test', accuracy_score(y_sex__, y_sex_test))
print('Train', accuracy_score(sex_predictor.predict(X_train), y_sex_train))

Test 0.624056014004
Train 0.653533560068


In [25]:
mean_ = np.zeros_like(y_sex_test)
print(accuracy_score(mean_, y_sex_test))

0.555938984746


In [27]:
age_predictor = LinearRegression()

age_predictor.fit(X_train, y_age_train)
y_age__ = age_predictor.predict(X_test)
y_age__ = np.exp(y_age__ + np.log(16))

print('Test', mean_absolute_error(y_age__, np.exp(y_age_test + np.log(16))))
print('Train', mean_absolute_error(np.exp(age_predictor.predict(X_train) + np.log(16)), np.exp(y_age_train + np.log(16))))

Test 5.87442500246
Train 5.67931056103


In [23]:
mean_ = np.full_like(y_age_test, 28)
print(mean_absolute_error(mean_, np.exp(y_age_test + np.log(16))))

6.24376094024
